In [74]:
import pandas as pd
import polars as pl
import tqdm
import numpy as np
from scipy.spatial.distance import cdist
import os

import geopandas as gpd

In [2]:
in_path = r"D:/Database/Trans_ontime_perf.csv"

data = pd.read_csv(in_path)

In [3]:
from functools import wraps
import time


def timeit(func):
    @wraps(func)
    def timeit_wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        total_time = end_time - start_time
        print(f'Function {func.__name__}{args} {kwargs} Took {total_time:.4f} seconds')
        return result
    return timeit_wrapper

In [110]:

def data_download(url_name,file_name,out_path):
    #lazy import
    from zipfile import ZipFile
    from urllib.request import urlopen   
    import os
    URL = url_name
    file_name = file_name
  
    url = urlopen(URL)
    output = open(file_name+'.zip', 'wb')    # note the flag:  "wb"

    output.write(url.read())
    output.close()
 
    # read the zip file as a pandas dataframe
    df = pd.read_csv(file_name+'.zip')   # pandas version 0.18.1 takes zip files     
    # df.to_csv(out_path+file_name)
    # if keeping on disk the zip file is not wanted, then:
 
    os.remove(file_name+'.zip')
    df.to_csv(out_path+file_name+'.csv')
    return

In [111]:
urls = data.URL
pbar = tqdm.tqdm(urls[2:])
out_path = r"D:/Database/on_time_data/"
for url in pbar:
    file_name = url.split('/')[-1].split('.')[0]
    pbar.set_description('Processing file: %s' %file_name)
    data_download(url, file_name,out_path)

Processing file: on_time_performance_2021_09:   0%|                                             | 0/26 [00:00<?, ?it/s]C:\Users\blhar\AppData\Local\Temp\ipykernel_4604\3670572368.py:16: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name+'.zip')   # pandas version 0.18.1 takes zip files
Processing file: on_time_performance_2021_11:   8%|██▊                                  | 2/26 [01:29<17:49, 44.56s/it]C:\Users\blhar\AppData\Local\Temp\ipykernel_4604\3670572368.py:16: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name+'.zip')   # pandas version 0.18.1 takes zip files
Processing file: on_time_performance_2022_06:  35%|████████████▊                        | 9/26 [06:41<12:21, 43.63s/it]C:\Users\blhar\AppData\Local\Temp\ipykernel_4604\3670572368.py:16: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set lo

# above this cell is downloading data
# below is transforming data

In [5]:
weather_path = r'D:\Database\winnipeg 2021-01-01 to 2023-11-30.csv'
traffic_path = r'D:\Database\Permanent_Count_Station_Traffic_Counts.csv'
bus_use_path = r'D:\Database\Estimated_Daily_Passenger_Activity.csv'
oct_data_path = r'D:\Database\on_time_data\on_time_performance_2023_10.csv'

In [6]:
weather = pd.read_csv(weather_path)
traffic = pd.read_csv(traffic_path)
bus_use = pd.read_csv(bus_use_path)


In [7]:
oct_data2 = pd.read_csv(oct_data_path)

In [8]:
traffic.Timestamp = pd.to_datetime(traffic.Timestamp)


In [9]:
traffic = traffic.sort_values(['Site','Timestamp']).reset_index(drop = True)
traffic2 = traffic.loc[:,['Timestamp','Site','Right','Left','Total','Latitude','Longitude']]
traffic2['date_key']= traffic2.Timestamp.dt.floor('h')

In [10]:
traffic3 = traffic2.groupby(['Site','date_key']).mean()
traffic3 = traffic3.reset_index()

In [11]:
weather2 = weather.loc[:,['windspeed','precip','temp','datetime', 'preciptype', 'snow', 'snowdepth']]
weather2['datetime'] = pd.to_datetime(weather2.datetime)


In [117]:
def load_trans(_data,w_data,t_data):
    _data['Scheduled Time'] = pd.to_datetime(_data['Scheduled Time'])
    _data['date_key'] = _data['Scheduled Time'].dt.floor('h')
    _data = _data.merge(w_data,how = 'left',left_on = 'date_key', right_on = 'datetime')
    _data['Longitude'] = _data.Location.apply(lambda x:float(x.split(' ')[1].split('(')[1]))
    _data['Latitude'] = _data.Location.apply(lambda x:float(x.split(' ')[2].split(')')[0]))
    
    area_map = t_data.loc[:,['Site','Latitude','Longitude']].groupby(['Site']).mean().reset_index().reset_index()
    temp = cdist(_data.loc[:,['Longitude','Latitude']],area_map [["Longitude","Latitude"]])
    _data['area_index'] = np.argmin(temp,axis =1)
    
    # for i,row in tqdm.tqdm(enumerate(temp)):
    #     _data.iloc[i,-1] = np.argmin(row)
    
    t_data = t_data.merge(area_map.loc[:,['index','Site']],how='left',left_on = 'Site',right_on = 'Site')
    t_data.date_key = pd.to_datetime(t_data.date_key).dt.tz_localize(None)
  
    _data = _data.merge(t_data,how = 'left',left_on=['area_index','date_key'], right_on = ['index','date_key'])
    _data = _data.drop(['Unnamed: 0','Row ID','Timestamp','index', 'temp','Location'],axis = 1)
    _data.preciptype = _data.preciptype.fillna('None')
    _data = _data.fillna(0)
    return _data

In [35]:
test = load_trans(oct_data2, weather2,traffic3)

In [120]:
file_list = os.listdir(r'D:\Database\on_time_data') 

In [121]:
file_list[-3:]

['v2_on_time_performance_2023_08.csv',
 'v2_on_time_performance_2023_09.csv',
 'v2_on_time_performance_2023_10.csv']

In [118]:
pbar = tqdm.tqdm(file_list[-3:])
for name in pbar:
    file_name = r'D:/Database/on_time_data/' + name
    raw_data = pd.read_csv(file_name)
    pbar.set_description('imported file: %s - Starting Processing'  %name)
    out_file = load_trans(raw_data, weather2,traffic3)
    out_file.to_csv( r'D:/Database/on_time_data/v2_' + name)
    del out_file

imported file: on_time_performance_2023_10.csv - Starting Processing: 100%|██████████████| 3/3 [04:56<00:00, 98.85s/it]


In [101]:
fields = ['Stop Number','Route Number','Route Name','Route Destination'
,'Day Type'
,'Scheduled Time'	
,'Deviation'
,'date_key'
,'windspeed'
,'precip'
,'preciptype'
,'Site'
,'Right'
,'Left'
,'Total']

In [ ]:
file_list[-3:]

C:\Users\blhar\AppData\Local\Temp\ipykernel_4604\3487216635.py:1: DtypeWarning: Columns (2,12) have mixed types. Specify dtype option on import or set low_memory=False.
  target_data = pd.read_csv(r'D:/Database/on_time_data/'+file_list[-3])


In [75]:
# Merging in area shapes file

area_shape = gpd.read_file(r'D:\Database\Neighbourhood.geojson')


In [134]:
area_shape.head()

,id,name,geometry
0,759,Rossmere-B,"MULTIPOLYGON (((-97.08240 49.92037, -97.08139 ..."
1,87,Glendale,"MULTIPOLYGON (((-97.32647 49.87836, -97.32679 ..."
2,1474,Armstrong Point,"MULTIPOLYGON (((-97.15236 49.87560, -97.15310 ..."
3,95,Burrows-Keewatin,"MULTIPOLYGON (((-97.18195 49.92749, -97.18201 ..."
4,158,Melrose,"MULTIPOLYGON (((-97.02536 49.89362, -97.02536 ..."


In [152]:
target_file = r'D:/Database/on_time_data/'+file_list[-3]
target_file

'D:/Database/on_time_data/v2_on_time_performance_2023_08.csv'

In [153]:

target_data = pd.read_csv(target_file)
target_data.shape
# target_data = target_data.drop(['Unnamed: 0'],axis = 1)
# target_data = gpd.GeoDataFrame(target_data,
#                                 geometry=gpd.points_from_xy(target_data.Longitude_x,target_data.Latitude_x	),crs="EPSG:4326")

C:\Users\blhar\AppData\Local\Temp\ipykernel_4604\427127472.py:1: DtypeWarning: Columns (2,12) have mixed types. Specify dtype option on import or set low_memory=False.
  target_data = pd.read_csv(target_file)


MemoryError: Unable to allocate 400. MiB for an array with shape (11, 4761400) and data type object

In [148]:
df = gpd.sjoin(target_data,area_shape,how = 'left')

In [151]:
df.shape

(4500909, 27)

In [150]:
df.to_csv(target_file)